In [188]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
import requests
import math
import csv
import datetime
import re

In [176]:
import xgboost
xgboost.__version__

'0.81'

In [ ]:
def encode_csv(path, outpath):
    with open(path, 'r', encoding='utf-8', errors='ignore') as infile, open(outpath, 'w') as outfile:
     inputs = csv.reader(infile)
     output = csv.writer(outfile)

     for index, row in enumerate(inputs):
         output.writerow(row)

In [152]:
import nltk
# nltk.download()
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [3]:
# MacOS garbage
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
encode_csv('./training_set.csv', './training_set_encoded.csv')

In [183]:
# Pickle
X.to_pickle("./cleaned_data.pkl")

In [184]:
# Load Pickle
Xp = pd.read_pickle("./cleaned_data.pkl")

In [162]:
train_df = pd.read_csv('./training_set_encoded.csv')
train_df.head()

,Engagements,Followers at Posting,Created,Type,Description
0,502093,36984682,2019-05-21 23:30:51 EDT,Video,The @raptors bench trio of @sergeibaka @norman...
1,603380,36984682,2019-05-21 22:53:33 EDT,Video,@kyle_lowry7 pulls from deep for the @raptors ...
2,603380,36984682,2019-05-21 22:19:58 EDT,Video,@k_mid22 with some english on the @bucks dime!
3,725100,36984682,2019-05-21 22:02:41 EDT,Video,Kawhi punches it home with the left on TNT!
4,661446,36984682,2019-05-21 20:47:49 EDT,Video,@giannis_an34 goes baseline early to rock the ...


In [163]:
train_sample_fraction = 0.2
y = train_df[['Engagements']]
X = train_df[['Followers at Posting']]
X.head()

,Followers at Posting
0,36984682
1,36984682
2,36984682
3,36984682
4,36984682


In [166]:
# Create features for the date data
rows = train_df.size
X['Year'] = pd.DataFrame(0, index=range(rows), columns=range(1))
X['Month'] = pd.DataFrame(0, index=range(rows), columns=range(1))
X['Day'] = pd.DataFrame(0, index=range(rows), columns=range(1))
X['Hour'] = pd.DataFrame(0, index=range(rows), columns=range(1))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [171]:
dates = train_df[['Created']]

for r in range(dates.values.size):
    row = dates.values[r].tolist()
    row = "".join(row).split()
    date = row[0].split('-')
    year = date[0]
    month = date[1]
    day = date[2]
    weekday = datetime.datetime(int(year), int(month), int(day)).weekday()
    hour = row[1].split(":")[0]
    X['Year'][r] = year
    X['Month'][r] = month
    X['Day'][r] = weekday
    X['Hour'][r] = hour

X.head(10)

,Followers at Posting,Year,Month,Day,Hour
0,36984682,2019,5,1,23
1,36984682,2019,5,1,22
2,36984682,2019,5,1,22
3,36984682,2019,5,1,22
4,36984682,2019,5,1,20
5,36955156,2019,5,1,14
6,36955156,2019,5,1,12
7,36955156,2019,5,1,8
8,36955156,2019,5,1,0
9,36955156,2019,5,1,0


In [173]:
# Create feature for the type data
Media_Type = train_df[['Type']]

label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(Media_Type)
Media_Type = label_encoder.transform(Media_Type)
label_encoder.classes_

X['Type'] = Media_Type
X.tail()

,Followers at Posting,Year,Month,Day,Hour,Type
7761,24875542,2017,10,6,19,2
7762,24875542,2017,10,6,13,2
7763,24875542,2017,10,6,9,0
7764,24875542,2017,10,6,2,2
7765,24875542,2017,10,6,0,2


In [174]:
# Create features for sentiment analysis
rows = train_df.size
X['neg'] = pd.DataFrame(0.0, index=range(rows), columns=range(1))
X['neu'] = pd.DataFrame(0.0, index=range(rows), columns=range(1))
X['pos'] = pd.DataFrame(0.0, index=range(rows), columns=range(1))
X['compound'] = pd.DataFrame(0.0, index=range(rows), columns=range(1))

In [175]:
# Sentiment analysis on "Description"
description = train_df[['Description']]

analyser = SentimentIntensityAnalyzer()

for r in range(description.values.size):
    row = description.values[r].tolist()
    if isinstance(row[0], str):
        snt = analyser.polarity_scores(row[0])
        X['neg'][r] = snt['neg']
        X['neu'][r] = snt['neu']
        X['pos'][r] = snt['pos']
        X['compound'][r] = snt['compound']

X.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/anaconda3/lib/python3.7/si

,Followers at Posting,Year,Month,Day,Hour,Type,neg,neu,pos,compound
0,36984682,2019,5,1,23,2,0.0,1.0,0.0,0.0
1,36984682,2019,5,1,22,2,0.0,1.0,0.0,0.0
2,36984682,2019,5,1,22,2,0.0,1.0,0.0,0.0
3,36984682,2019,5,1,22,2,0.0,1.0,0.0,0.0
4,36984682,2019,5,1,20,2,0.0,1.0,0.0,0.0


In [218]:
X['followers'] = pd.DataFrame(0, index=range(rows), columns=range(1))
X.head()

,Followers at Posting,Year,Month,Day,Hour,Type,neg,neu,pos,compound,followers
0,36984682,2019,5,1,23,2,0.0,1.0,0.0,0.0,0
1,36984682,2019,5,1,22,2,0.0,1.0,0.0,0.0,0
2,36984682,2019,5,1,22,2,0.0,1.0,0.0,0.0,0
3,36984682,2019,5,1,22,2,0.0,1.0,0.0,0.0,0
4,36984682,2019,5,1,20,2,0.0,1.0,0.0,0.0,0


In [219]:
def get_followers(user):
    url = 'https://www.instagram.com/' + user
    r = requests.get(url).text
    start = '"edge_followed_by":{"count":'
    end = '},"followed_by_viewer"'
    return r[r.find(start)+len(start):r.rfind(end)]

In [ ]:
for r in range(description.values.size):
    row = description.values[r].tolist()[0]
    if isinstance(row, str):
        handles = re.findall(r'@([A-Za-z0-9_]+)', row)
        sum = 0
        for handle in handles:
            try:
                sum += int(get_followers(handle))
            except:
                sum += 0
        X['followers'][r] = sum

X.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=train_sample_fraction, random_state=251)

In [ ]:
xgb = XGBRegressor()
params = {
    'eta': [0.15, 0.2, 0.25, 0.3, 0.35],
    'gamma': [0, 0.01, 0.05, 0.10],
    'lambda': [1, 0, 0.5, 1.5, 3],
    'alpha': [0, 1, 0.5, 1.5, 3]
}
model = GridSearchCV(xgb, param_grid = params, scoring = 'neg_mean_absolute_error')
model.fit(X_train, y_train)

In [ ]:
xgb = XGBRegressor()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    sum = 0
    for t, p in zip(y_true, y_pred):
        sum += abs((t - p)/t)
    sum /= len(y_true)
    
    return sum[0]*100

In [ ]:
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print(MAPE)
MAE = mean_absolute_error(y_test, y_pred)
print(MAE)

In [ ]:
encode_csv('./holdout_set.csv', './holdout_set_encoded.csv')

In [ ]:
test_df = pd.read_csv('./holdout_set_encoded.csv'')
test_df.head()

In [ ]:
X = test_df.drop(['Engagements'], axis=1)
X.head()

In [161]:
y_pred = xgb.predict(X)
train_df['Engagements'] = y_pred
with open("holdout_set_bad.csv", "w") as f:
    train_df.to_csv(f)

In [212]:
X = X.drop(['followers'], axis=1)
X

,Followers at Posting,Year,Month,Day,Hour,Type,neg,neu,pos,compound
0,36984682,2019,5,1,23,2,0.000,1.000,0.000,0.0000
1,36984682,2019,5,1,22,2,0.000,1.000,0.000,0.0000
2,36984682,2019,5,1,22,2,0.000,1.000,0.000,0.0000
3,36984682,2019,5,1,22,2,0.000,1.000,0.000,0.0000
4,36984682,2019,5,1,20,2,0.000,1.000,0.000,0.0000
5,36955156,2019,5,1,14,1,0.000,0.779,0.221,0.5707
6,36955156,2019,5,1,12,2,0.000,1.000,0.000,0.0000
7,36955156,2019,5,1,8,1,0.000,1.000,0.000,0.0000
8,36955156,2019,5,1,0,1,0.000,0.730,0.270,0.5707
9,36955156,2019,5,1,0,2,0.000,1.000,0.000,0.0000
